# H3-Toolkit Demo

This notebook demonstrates the features of the `h3-toolkit` library, including boundary tracing and GeoJSON generation.

In [1]:
import h3
import folium
import json
from h3_toolkit.utils import trace_cell_to_parent_faces
from h3_toolkit.geom import cell_boundary_to_geojson, get_boundary_cells

print("Libraries imported successfully.")

Libraries imported successfully.


## 1. Boundary Tracing

Let's pick a cell and trace its relationship to its parent's faces.

In [2]:
# Example H3 Cell (San Francisco)
h = h3.latlng_to_cell(37.775938728915946, -122.41795063018799, 9)

# Trace faces to parent
parent = h3.cell_to_parent(h, 8)
all_faces = {1, 2, 3, 4, 5, 6}
faces = trace_cell_to_parent_faces(h, all_faces)

print(f"Cell: {h}")
print(f"Parent: {parent}")
print(f"Lies on parent faces: {faces}")

Cell: 8928308280fffff
Parent: 8828308281fffff
Lies on parent faces: {2, 3}


## 2. GeoJSON Visualization

We can convert the cell to GeoJSON and visualize it on a map.

In [3]:
feature = cell_boundary_to_geojson(h)
feature_str = json.dumps(feature, indent=2)
print(feature_str)

{
  "type": "Feature",
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -122.4172,
          37.775198
        ],
        [
          -122.416128,
          37.77688
        ],
        [
          -122.417388,
          37.778385
        ],
        [
          -122.419719,
          37.778207
        ],
        [
          -122.42079,
          37.776524
        ],
        [
          -122.419531,
          37.77502
        ],
        [
          -122.4172,
          37.775198
        ]
      ]
    ]
  },
  "properties": {
    "h3_index": "8928308280fffff"
  }
}


In [4]:
m = folium.Map(location=[37.775938728915946, -122.41795063018799], zoom_start=15)

# Add cell boundary
folium.GeoJson(
    feature,
    style_function=lambda x: {'color': 'blue', 'fillOpacity': 0.1, 'weight': 2}
).add_to(m)

# Add parent boundary for context
parent_feature = cell_boundary_to_geojson(parent)
folium.GeoJson(
    parent_feature,
    style_function=lambda x: {'color': 'red', 'fillOpacity': 0.05, 'weight': 4, 'dashArray': '5, 5'}
).add_to(m)

m

## 3. Boundary Coverage

Identify cells that cover a random polygon boundary.

In [11]:
# Create a simple polygon around the point
polygon = {
    "type": "Polygon",
    "coordinates": [[
        [-122.42, 37.776],
        [-122.415, 37.776],
        [-122.415, 37.772],
        [-122.42, 37.772],
        [-122.42, 37.776]
    ]]
}

res = 11
boundary_cells = get_boundary_cells(polygon, res)

print(f"Found {len(boundary_cells)} boundary cells at resolution {res}")

Found 33 boundary cells at resolution 11


In [12]:
m2 = folium.Map(location=[37.774, -122.4175], zoom_start=16)

# Polygon
folium.GeoJson(
    polygon,
    style_function=lambda x: {'color': 'green', 'fillColor': 'green', 'fillOpacity': 0.1}
).add_to(m2)

# Boundary cells
for cell in boundary_cells:
    cell_feature = cell_boundary_to_geojson(cell)
    folium.GeoJson(
        cell_feature,
        style_function=lambda x: {'color': 'orange', 'weight': 1, 'fillOpacity': 0.3}
    ).add_to(m2)

m2